## Dataset

In [1]:
from datasets import load_dataset

data_ljp_criminal = load_dataset("lbox/lbox_open", "ljp_criminal")

data_ljp_criminal

Found cached dataset lbox_open (/home/yunho/.cache/huggingface/datasets/lbox___lbox_open/ljp_criminal/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae)


  0%|          | 0/4 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'casetype', 'casename', 'facts', 'reason', 'label', 'ruling'],
        num_rows: 8400
    })
    validation: Dataset({
        features: ['id', 'casetype', 'casename', 'facts', 'reason', 'label', 'ruling'],
        num_rows: 1050
    })
    test: Dataset({
        features: ['id', 'casetype', 'casename', 'facts', 'reason', 'label', 'ruling'],
        num_rows: 1050
    })
    test2: Dataset({
        features: ['id', 'casetype', 'casename', 'facts', 'reason', 'label', 'ruling'],
        num_rows: 928
    })
})

In [2]:
train_data = data_ljp_criminal['train']
train_data[0]

{'id': 0,
 'casetype': 'criminal',
 'casename': '강제추행',
 'facts': '피고인은 2020. 9. 25. 23:45경 포항시 남구 B에 있는 피해자 C(여, 62세) 운영의 D에서 피해자와 함께 술을 마신 후 결제를 하고 나왔고, 피해자가 피고인을 배웅하자 갑자기 피해자의 양 손목을 잡고 포항시 남구 E에 있는 F 출입구까지 약 20m 가량 끌고 간 뒤, 피해자를 강제로 끌어안고 입을 맞추며 손으로 피해자의 가슴을 수회 만졌다.\n이로써 피고인은 피해자를 강제로 추행하였다.',
 'reason': '양형의 이유\n1. 법률상 처단형의 범위: 징역 1월 ∼ 10년\n2. 양형기준에 따른 권고형의 범위\n[유형의 결정] 성범죄 > 01. 일반적 기준 > 나. 강제추행죄(13세 이상 대상) > [제1유형] 일반강제추행\n[특별양형인자] 감경요소 : 처벌불원\n[권고영역 및 권고형의 범위] 감경영역, 징역 1월∼1년\n3. 선고형의 결정 :\n추행의 정도가 가볍지 않은 점을 고려하여 징역형으로 처벌하되, 피고인이 이 사건 범행을 인정하고 있는 점, 피해자와 합의하여 피해자가 피고인의 처벌을 원하지 않는 점, 그 밖에 피고인의 나이, 성행, 환경, 범행의 동기와 경위, 범행 후의 정황 등 이 사건 변론에 나타난 여러 양형조건을 종합하여 주문과 같이 형을 정한다.\n신상정보 등록 및 제출의무\n판시 범죄사실에 관한 유죄판결이 확정되는 경우 피고인은 성폭력범죄의 처벌 등에 관한 특례법 제42조 제1항에 의하여 신상정보 등록대상자에 해당하게 되므로, 같은 법 제43조에 따라 관할기관에 신상정보를 제출할 의무가 있다.',
 'label': {'text': '징역 6월',
  'fine_lv': 0,
  'imprisonment_with_labor_lv': 2,
  'imprisonment_without_labor_lv': 0},
 'ruling': {'text': '피고인을 징역 6월에 처한다.\n다만, 이 판결 확정일부터 2년간

- id: a data id.
- casetype: a case type. The value is always criminal.
- casename: a case name.
- facts: facts (사실관계) extracted from reasoning (이유) section of individual cases.
- label
    - fine_lv: a label representing individual ranges of the fine amount. See our paper for the detail.
    - imprisonment_with_labor_lv: a label representing the ranges of the imprisonemnt with labor.
    - imprisonment_without_labor_lv: a label for the imprisonment without labor case.
- reason: the reason for the punishment (양형의 이유).
- ruling: the ruling (주문) and its parsing result. "" and -1 indicates null values.

| instruction | input | output |
| --- | --- | --- |
| casename을 맞추세요 | facts | casename(강제추행 등) |
| --- | --- | --- |
| 판결 이유를 설명하세요 | (casetype) + facts | reason |
| --- | --- | --- |
| 판결을 예측해보세요 | facts + reason | label // ruling(text, parse) |
| 벌금을 예측해보세요 | facts + reason | fine_lv // ruling(parse-fine)|
| 징역을 예측해보세요 | facts + reason | imprisonment_without_labor_lv // ruling(pares-imprisonment) |

In [3]:
from collections import Counter

Counter(train_data['casename'])

Counter({'강제추행': 1200,
         '공무집행방해': 1200,
         '교통사고처리특례법위반(치상)': 1200,
         '도로교통법위반(음주운전)': 1200,
         '사기': 1200,
         '상해': 1200,
         '폭행': 1200})

In [4]:
# casename from input
casename_dataset = []
for data in train_data:
    new_data = {}
    new_data['instruction'] = '사건 종류를 분류하시오.'
    new_data['input'] = data['facts']
    new_data['output'] = data['casename']
    casename_dataset.append(new_data)
    
print(casename_dataset[:5])
print(casename_dataset[-5:])
print(len(casename_dataset))

[{'instruction': '사건 종류를 분류하시오.', 'input': '피고인은 2020. 9. 25. 23:45경 포항시 남구 B에 있는 피해자 C(여, 62세) 운영의 D에서 피해자와 함께 술을 마신 후 결제를 하고 나왔고, 피해자가 피고인을 배웅하자 갑자기 피해자의 양 손목을 잡고 포항시 남구 E에 있는 F 출입구까지 약 20m 가량 끌고 간 뒤, 피해자를 강제로 끌어안고 입을 맞추며 손으로 피해자의 가슴을 수회 만졌다.\n이로써 피고인은 피해자를 강제로 추행하였다.', 'output': '강제추행'}, {'instruction': '사건 종류를 분류하시오.', 'input': '피고인은 2019. 9.5. 08:16경 9호선 전동차를 타고 가던 중, 서울 서초구 B에 있는 C역에서 하차하면서 옆에 있던 피해자 D(가명, 여, 30세)의 음부 부분을 손등으로 갑자기 1회 만져 피해자를 강제로 추행하였다.', 'output': '강제추행'}, {'instruction': '사건 종류를 분류하시오.', 'input': '피고인은 2019. 8. 19. 23:00경 서울 강서구 C 오피스텔 D동 1층 입구에서 피해자 E(가명, 여, 25세)에게 전화번호를 알려 달라고 요구하였으나, 피해자가 이를 거부하자 "너가 뭐가 그렇게 잘났어, 너가 그렇게 대단해. "라고 말하면서 갑자기 손으로 피해자의 손목을 잡고, 피해자의 오른쪽 가슴을 움켜쥐어 피해자를 강제로 추행하였다.', 'output': '강제추행'}, {'instruction': '사건 종류를 분류하시오.', 'input': '피고인은 2021. 10. 5. 01:00경 피해자 B(가명, 여, 24세)의 주거지인 논산시 C에 있는 오피스텔에서, 방바닥에서 잠을 자다가 일어나 피해자가 누워있는 침대에 올라가 갑자기 피해자의 몸 뒤쪽에 밀착하고 누워 약 3분간 피해자를 껴안고 엉덩이에 성기를 비벼 피해자를 강제로 추행하였다.', 'output': '강제추행'}, {'instruction': '사건 종류를 

In [5]:
# reason from fact
reason_dataset = []
for data in train_data:
    new_data = {}
    new_data['instruction'] = '판결 이유를 설명하시오.'
    new_data['input'] = "사건 종류: " + data['casename'] + ", 법률사실: " + data['facts']
    new_data['output'] = data['reason']
    reason_dataset.append(new_data)
    
print(reason_dataset[:5])
print(reason_dataset[-5:])
print(len(reason_dataset))

[{'instruction': '판결 이유를 설명하시오.', 'input': '사건 종류: 강제추행, 법률사실: 피고인은 2020. 9. 25. 23:45경 포항시 남구 B에 있는 피해자 C(여, 62세) 운영의 D에서 피해자와 함께 술을 마신 후 결제를 하고 나왔고, 피해자가 피고인을 배웅하자 갑자기 피해자의 양 손목을 잡고 포항시 남구 E에 있는 F 출입구까지 약 20m 가량 끌고 간 뒤, 피해자를 강제로 끌어안고 입을 맞추며 손으로 피해자의 가슴을 수회 만졌다.\n이로써 피고인은 피해자를 강제로 추행하였다.', 'output': '양형의 이유\n1. 법률상 처단형의 범위: 징역 1월 ∼ 10년\n2. 양형기준에 따른 권고형의 범위\n[유형의 결정] 성범죄 > 01. 일반적 기준 > 나. 강제추행죄(13세 이상 대상) > [제1유형] 일반강제추행\n[특별양형인자] 감경요소 : 처벌불원\n[권고영역 및 권고형의 범위] 감경영역, 징역 1월∼1년\n3. 선고형의 결정 :\n추행의 정도가 가볍지 않은 점을 고려하여 징역형으로 처벌하되, 피고인이 이 사건 범행을 인정하고 있는 점, 피해자와 합의하여 피해자가 피고인의 처벌을 원하지 않는 점, 그 밖에 피고인의 나이, 성행, 환경, 범행의 동기와 경위, 범행 후의 정황 등 이 사건 변론에 나타난 여러 양형조건을 종합하여 주문과 같이 형을 정한다.\n신상정보 등록 및 제출의무\n판시 범죄사실에 관한 유죄판결이 확정되는 경우 피고인은 성폭력범죄의 처벌 등에 관한 특례법 제42조 제1항에 의하여 신상정보 등록대상자에 해당하게 되므로, 같은 법 제43조에 따라 관할기관에 신상정보를 제출할 의무가 있다.'}, {'instruction': '판결 이유를 설명하시오.', 'input': '사건 종류: 강제추행, 법률사실: 피고인은 2019. 9.5. 08:16경 9호선 전동차를 타고 가던 중, 서울 서초구 B에 있는 C역에서 하차하면서 옆에 있던 피해자 D(가명, 여, 30세)의 음부 부분을 손등으로 갑자기 1회 만져 피해자를 

In [6]:
# sentence from fact
sentence_dataset = []
for data in train_data:
    new_data = {}
    new_data['instruction'] = '재판 결과를 예측해보시오.'
    new_data['input'] = "법률사실: " + data['facts'] + ", 판결 요지: " + data['reason']
    new_data['output'] = data['ruling']['text']
    sentence_dataset.append(new_data)
    
print(sentence_dataset[:5])
print(sentence_dataset[-5:])
print(len(sentence_dataset))

[{'instruction': '재판 결과를 예측해보시오.', 'input': '법률사실: 피고인은 2020. 9. 25. 23:45경 포항시 남구 B에 있는 피해자 C(여, 62세) 운영의 D에서 피해자와 함께 술을 마신 후 결제를 하고 나왔고, 피해자가 피고인을 배웅하자 갑자기 피해자의 양 손목을 잡고 포항시 남구 E에 있는 F 출입구까지 약 20m 가량 끌고 간 뒤, 피해자를 강제로 끌어안고 입을 맞추며 손으로 피해자의 가슴을 수회 만졌다.\n이로써 피고인은 피해자를 강제로 추행하였다., 판결 요지: 양형의 이유\n1. 법률상 처단형의 범위: 징역 1월 ∼ 10년\n2. 양형기준에 따른 권고형의 범위\n[유형의 결정] 성범죄 > 01. 일반적 기준 > 나. 강제추행죄(13세 이상 대상) > [제1유형] 일반강제추행\n[특별양형인자] 감경요소 : 처벌불원\n[권고영역 및 권고형의 범위] 감경영역, 징역 1월∼1년\n3. 선고형의 결정 :\n추행의 정도가 가볍지 않은 점을 고려하여 징역형으로 처벌하되, 피고인이 이 사건 범행을 인정하고 있는 점, 피해자와 합의하여 피해자가 피고인의 처벌을 원하지 않는 점, 그 밖에 피고인의 나이, 성행, 환경, 범행의 동기와 경위, 범행 후의 정황 등 이 사건 변론에 나타난 여러 양형조건을 종합하여 주문과 같이 형을 정한다.\n신상정보 등록 및 제출의무\n판시 범죄사실에 관한 유죄판결이 확정되는 경우 피고인은 성폭력범죄의 처벌 등에 관한 특례법 제42조 제1항에 의하여 신상정보 등록대상자에 해당하게 되므로, 같은 법 제43조에 따라 관할기관에 신상정보를 제출할 의무가 있다.', 'output': '피고인을 징역 6월에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 집행을 유예한다.\n피고인에게 40시간의 성폭력치료강의 수강을 명한다.'}, {'instruction': '재판 결과를 예측해보시오.', 'input': '법률사실: 피고인은 2019. 9.5. 08:16경 9호선 전동차를 타고 가던 중, 서울 서초구 B에 

In [7]:
import os

PATH = os.getcwd()
PATH

'/home/yunho/Documents/NLP/legal_prediction/chatbot'

In [9]:
import json

with open(PATH + '/Data/casename_dataset.json', 'w', encoding='utf-8') as json_file:
    json.dump(casename_dataset, json_file, ensure_ascii=False, indent=4)

with open(PATH + '/Data/reason_dataset.json', 'w', encoding='utf-8') as json_file:
   json.dump(reason_dataset, json_file, ensure_ascii=False, indent=4)

with open(PATH + '/Data/sentence_dataset.json', 'w', encoding='utf-8') as json_file:
    json.dump(sentence_dataset, json_file, ensure_ascii=False, indent=4)

## LLaMA

In [10]:
!git clone -q https://github.com/kairess/alpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt -q
!pip uninstall peft -y -q
!pip install -q git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

fatal: destination path 'alpaca-lora' already exists and is not an empty directory.
/home/yunho/Documents/NLP/legal_prediction/chatbot/alpaca-lora


In [11]:
import json

prompt_template = {
    "description": "Alpaca-LoRA Custom 템플릿",
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response:\n"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response:\n"
    ),
    "response_split": "### Response:",
}

with open('templates/custom.json', 'w', encoding='utf-8') as f:
    json.dump(prompt_template, f, ensure_ascii=False)

In [12]:
!pwd

/home/yunho/Documents/NLP/legal_prediction/chatbot/alpaca-lora


In [14]:
!python finetune.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --data_path '../Data/casename_dataset.json' \
    --output_dir './output' \
    --num_epochs 10 \
    --learning_rate 5e-4 \
    --val_set_size 1680 \
    --batch_size 1024 \
    --micro_batch_size 128 \
    --prompt_template_name 'custom'

Training Alpaca-LoRA model with params:
base_model: decapoda-research/llama-7b-hf
data_path: ../Data/casename_dataset.json
output_dir: ./output
batch_size: 1024
micro_batch_size: 128
num_epochs: 10
learning_rate: 0.0005
cutoff_len: 256
val_set_size: 1680
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
add_eos_token: False
group_by_length: False
wandb_project: 
wandb_run_name: 
wandb_watch: 
wandb_log_model: 
resume_from_checkpoint: False
prompt template: custom

Loading checkpoint shards: 100%|████████████████| 33/33 [00:07<00:00,  4.61it/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Found cached dataset json (/home/yunho/.cache/huggingface/datasets/json/default-0f7149f0ad8c7def/0.0.0/